# Introduction (688 only)
In this question, you'll build a basic recommendation system using collaborative filtering to make recommendations on a typical recommendation systems dataset, the MovieLens dataset. The purpose of this question is to become familiar with the internals of recommendation systems: both how they train and how they form recommendations. 

### Grading 
Your submission will be scored in the following manner: 
* process - 10pts
* train - 15pts
* recommend - 10pts

## Collaborative Filtering by Matrix Factorization
In collaborative filtering we wish to factorize our ratings matrix into two smaller feature matrices whose product is equal to the original ratings matrix. Specifically, given some partially filled ratings matrix $X\in \mathbb{R}^{m\times n}$, we want to find feature matrices $U \in \mathbb{R}^{m\times k}$ and $V \in \mathbb{R}^{n\times k}$ such that $UV^T = X$. In the case of movie recommendation, each row of $U$ could be features corresponding to a user, and each row of $V$ could be features corresponding to a movie, and so $u_i^Tv_j$ is the predicted rating of user $i$ on movie $j$. This forms the basis of our hypothesis function for collaborative filtering: 

$$h_\theta(i,j) = u_i^T v_j$$

In general, $X$ is only partially filled (and usually quite sparse), so we can indicate the non-presence of a rating with a 0. Notationally, let $S$ be the set of $(i,j)$ such that $X_{i,j} \neq 0$, so $S$ is the set of all pairs for which we have a rating. The loss used for collaborative filtering is squared loss:

$$\ell(h_\theta(i,j),X_{i,j}) = (h_\theta(i,j) - X_{i,j})^2$$

The last ingredient to collaborative filtering is to impose an $l_2$ weight penalty on the parameters, so our total loss is:

$$\sum_{i,j\in S}\ell(h_\theta(i,j),X_{i,j}) + \lambda_u ||U||_2^2 + \lambda_v ||V||_2^2$$

For this assignment, we'll let $\lambda_u = \lambda_v = \lambda$. 

## MovieLens rating dataset
To start off, let's get the MovieLens dataset. This dataset is actually quite large (24+ million ratings), but we will instead use their smaller subset of 100k ratings. You will have to go fetch this from their website. 

* You can download the archive containing their latest dataset release from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip (last updated October 2016). 
* For more details (contents and structure of archive), you can read the README at http://files.grouplens.org/datasets/movielens/ml-latest-README.html
* You can find general information from their website description located at http://grouplens.org/datasets/movielens/. 

For this assignment, we will only be looking at the ratings data specifically. However, it is good to note that there is additional data available (i.e. movie data and user made tags for movies) which could be used to improve the ability of the recommendation system. 

In [5]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg as la
import matplotlib
matplotlib.use("svg")
# AUTOLAB_IGNORE_START
%matplotlib inline
# AUTOLAB_IGNORE_STOP
import matplotlib.pyplot as plt
plt.style.use("ggplot")

C:\Users\gandh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\gandh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\gandh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\gandh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\gandh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\gandh\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\U

In [6]:
# AUTOLAB_IGNORE_START
movies = pd.read_csv("ml-latest-small/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
ratings.head()
# AUTOLAB_IGNORE_STOP

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
X=ratings.groupby(['userId', 'movieId']).sum().unstack()
X=X['rating']
X=X.fillna(0)
a=X.as_matrix()
a.shape
movieNames=list(movies['title'])
movieNames[:5]
X.shape


(671, 9066)

In [99]:
df=pd.merge(ratings, movies, on='movieId')
df.head()
df=df.sort_values(['userId','movieId'], axis=0).reindex()
len(df.movieId.unique())
rp = df.pivot_table(columns=['movieId'],index=['userId'],values='rating')
rp = rp.fillna(0); # Replace NaN
rp.head()
u=list(df.movieId.unique())
#print(df.iloc[1,:])

for i in rp.columns:
    if i not in u:
        rp=rp.drop(i,axis=1)
rp.shape

(671, 9066)

In [105]:
rp

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
import math
p=np.random.permutation(len(ratings))
p_tr=p[:math.floor(len(p)*0.9)]
p_te=p[math.floor(len(p)*0.9):]
#X2 = np.asarray(sp.coo_matrix((ratings["rating"][p_tr], (ratings["userId"]-1, ratings["movieId"]-1))).todense())
len(p_tr)+len(p_te)==len(p)

True

In [163]:
a_tr=np.zeros((rp.shape))
a_te=np.zeros((rp.shape))
p_tr
f=rp.values
f.shape
df2=df.reset_index(drop=True)
len(df2)
df3=df2.iloc[p_tr]
df3.shape
print(df3.iloc[0])
df3.userId[0]
df2
u

userId                                          378
movieId                                       91500
rating                                            1
timestamp                                1443625827
title                       The Hunger Games (2012)
genres       Action|Adventure|Drama|Sci-Fi|Thriller
Name: 51048, dtype: object


[31,
 1029,
 1061,
 1129,
 1172,
 1263,
 1287,
 1293,
 1339,
 1343,
 1371,
 1405,
 1953,
 2105,
 2150,
 2193,
 2294,
 2455,
 2968,
 3671,
 10,
 17,
 39,
 47,
 50,
 52,
 62,
 110,
 144,
 150,
 153,
 161,
 165,
 168,
 185,
 186,
 208,
 222,
 223,
 225,
 235,
 248,
 253,
 261,
 265,
 266,
 272,
 273,
 292,
 296,
 300,
 314,
 317,
 319,
 339,
 349,
 350,
 356,
 357,
 364,
 367,
 370,
 371,
 372,
 377,
 382,
 405,
 410,
 454,
 457,
 468,
 474,
 480,
 485,
 497,
 500,
 508,
 509,
 515,
 527,
 537,
 539,
 550,
 551,
 552,
 585,
 586,
 587,
 588,
 589,
 590,
 592,
 593,
 616,
 661,
 720,
 60,
 247,
 267,
 318,
 355,
 595,
 736,
 778,
 866,
 1197,
 1210,
 1235,
 1271,
 1378,
 1580,
 1721,
 1884,
 2028,
 2318,
 2513,
 2694,
 2702,
 2716,
 2762,
 2841,
 2858,
 2959,
 3243,
 3510,
 3949,
 5349,
 5669,
 6377,
 7153,
 7361,
 8622,
 8636,
 27369,
 44191,
 48783,
 50068,
 58559,
 84236,
 34,
 112,
 141,
 173,
 260,
 289,
 329,
 380,
 431,
 434,
 435,
 440,
 442,
 464,
 541,
 594,
 596,
 610,
 858,
 90

In [170]:
df2

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
5,1,1263,2.0,1260759151,"Deer Hunter, The (1978)",Drama|War
6,1,1287,2.0,1260759187,Ben-Hur (1959),Action|Adventure|Drama
7,1,1293,2.0,1260759148,Gandhi (1982),Drama
8,1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
9,1,1343,2.0,1260759131,Cape Fear (1991),Thriller


In [168]:
for i in p_tr:
    v1=df2.userId.iloc[i]-1
    v2=u.index(df2.movieId.iloc[i])
    v3=df2.rating[i]
    a_tr[v1,v2]=v3

In [175]:
for j in p_te:
    v4=df2.userId.iloc[j]-1
    v5=u.index(df2.movieId.iloc[j])
    v6=df2.rating[j]
    a_te[v4,v5]=v6

In [187]:
uu=list(df2.movieId)
uu.index(1069)
mn=[]

for n in u:
    mi=uu.index(n)
    mn.append(df2.title.iloc[mi])
mn

['Dangerous Minds (1995)',
 'Dumbo (1941)',
 'Sleepers (1996)',
 'Escape from New York (1981)',
 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)',
 'Deer Hunter, The (1978)',
 'Ben-Hur (1959)',
 'Gandhi (1982)',
 "Dracula (Bram Stoker's Dracula) (1992)",
 'Cape Fear (1991)',
 'Star Trek: The Motion Picture (1979)',
 'Beavis and Butt-Head Do America (1996)',
 'French Connection, The (1971)',
 'Tron (1982)',
 'Gods Must Be Crazy, The (1980)',
 'Willow (1988)',
 'Antz (1998)',
 'Fly, The (1986)',
 'Time Bandits (1981)',
 'Blazing Saddles (1974)',
 'GoldenEye (1995)',
 'Sense and Sensibility (1995)',
 'Clueless (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'Mighty Aphrodite (1995)',
 "Mr. Holland's Opus (1995)",
 'Braveheart (1995)',
 'Brothers McMullen, The (1995)',
 'Apollo 13 (1995)',
 'Batman Forever (1995)',
 'Crimson Tide (1995)',
 'Die Hard: With a Vengeance (1995)',
 'First Knight (1995)',
 'Net, The (1995)',
 'Nine Months (1995)',
 'Waterworld (1995)',
 

In [188]:
len(mn)
# qi=[]
# qm=[]
# for k in range(len(df2)):
#     mi=df2.movieId.iloc[k]
    
#     mn=

9066

## Data preparation

Matrix factorization requires that we have our ratings stored in a matrix of users, so the first task is to take the dataframe and convert it into this format. Note that in general, typically these matrices are extremely large and sparse (especially if you want to process the 24 million ratings), however for the purposes of this assignment a dense representation of this smaller dataset should fit on any machine. 

### Specification
* Split the data by assigning the first $\mathrm{floor}(9n/10)$ permuted entries to be the training set, and the remaining to be the testing set. Use the order given by the permutation. 
* Each row of the ratings matrix corresponds to a user. The first row of the matrix should correspond to the first user (by userID), and so on. This is because the set of user IDs already form a consecutive range of numbers. 
* Each column of the ratings matrix corresponds to a movie. The order of the columns doesn't matter, so long as the resulting list of movie names is accurate. This is because the set of movie IDs does not form a consecutive range of numbers. 
* Each user and movie that exists in the **ratings** dataframe should be present in the ratings matrix, even if it doesn't have any entries. We will only use the movies dataframe to extract the names of the movies. 
* Any entry that does not have a rating should have a default value of 0. 

In [189]:
import math
def process(ratings, movies, P):
    """ Given a dataframe of ratings and a random permutation, split the data into a training 
        and a testing set, in matrix form. 
        
        Args: 
            ratings (dataframe) : dataframe of MovieLens ratings
            movies (dataframe) : dataframe of MovieLens movies
            P (numpy 1D array) : random permutation vector
            
        Returns: 
            (X_tr, X_te, movie_names)  : training and testing splits of the ratings matrix (both 
                                         numpy 2D arrays), and a python list of movie names 
                                         corresponding to the columns of the ratings matrices. 
    """
    df=pd.merge(ratings, movies, on='movieId')
    df=df.sort_values(['userId','movieId'], axis=0).reindex()
    df2=df.reset_index(drop=True)
    
    u=list(df.movieId.unique())
    x=len(df.userId.unique())
    y=len(u)
    
    a_tr=np.zeros((x,y))
    a_te=np.zeros((x,y))
    
    p_tr=P[:math.floor(len(P)*0.9)]
    p_te=P[math.floor(len(P)*0.9):]  
    
    for i in p_tr:
        v1=df2.userId.iloc[i]-1
        v2=u.index(df2.movieId.iloc[i])
        v3=df2.rating[i]
        a_tr[v1,v2]=v3
        
    for j in p_te:
        v4=df2.userId.iloc[j]-1
        v5=u.index(df2.movieId.iloc[j])
        v6=df2.rating[j]
        a_te[v4,v5]=v6
        
    movie_names=[]
    uu=list(df2.movieId)
    
    #mn=[]

    for n in u:
        mi=uu.index(n)
        movie_names.append(df2.title.iloc[mi])
    
    return a_tr, a_te, movie_names

    
# AUTOLAB_IGNORE_START
X_tr, X_te, movieNames = process(ratings, movies, np.random.permutation(len(ratings)))
print(X_tr.shape, X_te.shape, movieNames[:5])
# AUTOLAB_IGNORE_STOP

(671, 9066) (671, 9066) ['Dangerous Minds (1995)', 'Dumbo (1941)', 'Sleepers (1996)', 'Escape from New York (1981)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)']


For example, running this on the small MovieLens dataset using a random permutation gives the following result: 
    
    (671L, 9066L) (671L, 9066L) ['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)', 'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)']

Your actual titles may vary depending on the random permutation given. 

## Alternating Minimization for Collaborative Filtering
Now we build the collaborative filtering recommendation system. We will use a method known as alternating least squares. Essentially, we will alternate between optimizing over $U$ and $V$ by holding the other constant. By treating one matrix as a constant, we get exactly a weighted least squares problem, which has a well-known solution. More details can be found in the lecture notes. 

### Specification
* Similar to the softmax regression on MNIST, there is a verbose parameter here that you can use to print your training err and test error. These should decrease (and converge). 
* You can assume a dense representation of all the inputs. 
* You may find it useful to have an indicator matrix W where $W_{ij} = 1$ if there is a rating in $X_{ij}$. 
* You can initialize U,V with random values. 

In [ ]:
def error(X, U, V):
    """ Compute the mean error of the observed ratings in X and their estimated values. 
        Args: 
            X (numpy 2D array) : a ratings matrix as specified above
            U (numpy 2D array) : a matrix of features for each user
            V (numpy 2D array) : a matrix of features for each movie
        Returns: 
            (float) : the mean squared error of the observed ratings with their estimated values
        """
    pass

def train(X, X_te, k, U, V, niters=51, lam=10, verbose=False):
    """ Train a collaborative filtering model. 
        Args: 
            X (numpy 2D array) : the training ratings matrix as specified above
            X_te (numpy 2D array) : the testing ratings matrix as specified above
            k (int) : the number of features use in the CF model
            U (numpy 2D array) : an initial matrix of features for each user
            V (numpy 2D array) : an initial matrix of features for each movie
            niters (int) : number of iterations to run
            lam (float) : regularization parameter
            verbose (boolean) : verbosity flag for printing useful messages
            
        Returns:
            (U,V) : A pair of the resulting learned matrix factorization
    """
    pass



Training the recommendation system with a random initialization of U,V with 5 features and $\lambda = 10$ results in the following output. Your results may vary depending on your random permutation.  

    Iter |Train Err |Test Err  
        0|    1.3854|    2.1635
        5|    0.7309|    1.5782
       10|    0.7029|    1.5078
       15|    0.6951|    1.4874
       20|    0.6910|    1.4746
       25|    0.6898|    1.4679
       30|    0.6894|    1.4648
       35|    0.6892|    1.4634
       40|    0.6891|    1.4631
       45|    0.6891|    1.4633
       50|    0.6891|    1.4636
    Wall time: 7min 32s

In [ ]:
# AUTOLAB_IGNORE_START
# AUTOLAB_IGNORE_STOP

## Recommendations

Finally, we need to be able to make recommendations given a matrix factorization. We can do this by simply taking the recommending the movie with the highest value in the estimated ratings matrix. 

### Specification
* For each user, recommend the the movie with the highest predicted rating for that user that the user **hasn't** seen before. 
* Return the result in a list such that the ith element in this list is the recommendation for the user corresponding to the ith row of the ratings matrix. 

In [ ]:
def recommend(X, U, V, movieNames):
    """ Recommend a new movie for every user.
        Args: 
            X (numpy 2D array) : the training ratings matrix as specified above
            U (numpy 2D array) : a learned matrix of features for each user
            V (numpy 2D array) : a learned matrix of features for each movie
            movieNames : a list of movie names corresponding to the columns of the ratings matrix
        Returns
            (list) : a list of movie names recommended for each user
    """
    pass
    
# AUTOLAB_IGNORE_START
recommendations = recommend(X_tr, U, V, movieNames)
print(recommendations[:10])
# AUTOLAB_IGNORE_STOP

Our implementation gets the following results (we can see they are all fairly popular and well known movies that were recommended). Again your results will vary depending on the random permutation. 

    ['Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Shawshank Redemption, The (1994)', 'Godfather, The (1972)', 'Fargo (1996)', 'Godfather, The (1972)', "Schindler's List (1993)"]